# Neural network Tuning

In [1]:
# Importing necesary packages
import numpy
import pandas as pd
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
import warnings
warnings.filterwarnings('ignore')
from keras.wrappers.scikit_learn import KerasClassifier

# part 1 - Tuning batch and epoch

In [2]:
# load the dataset for India diabetes
data = pd.read_csv('diabetes.csv')

In [ ]:
#viewing data
data.head()

In [3]:
# split into predictors and response
X = data.iloc[:,0:8] #predictors
Y = data.iloc[:,8] #response

In [ ]:
# Function to create model,for KerasClassifier
def create_my_model():
    #defining my model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))

    # Compile the model
    mymodel.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return mymodel

In [ ]:
# create model
model = KerasClassifier(build_fn=create_my_model)

In [ ]:
# define the grid search parameters
batchSize = [10, 20, 40, 60, 80, 100]
epochs = [10, 30, 50]

In [ ]:
parameter_grid = dict(batch_size=batchSize, epochs=epochs)


mygrid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = mygrid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# part 2- Optimizing Activation Function

In [ ]:
# Function to create model
def create_my_model(optimizer='adam'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation='relu'))
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [ ]:
# create model
model = KerasClassifier(build_fn=create_my_model, epochs=50, batch_size=10)

In [ ]:
# define the grid search parameters
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [ ]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

In [ ]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# How can you improve accuracy of the model?

In [97]:
from tensorflow.keras.layers import BatchNormalization

def create_my_model(optimizer='adam', activation='relu'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation=activation))
    BatchNormalization()
    mymodel.add(Dense(8, activation=activation))
    BatchNormalization()
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [120]:
model = KerasClassifier(build_fn=create_my_model, epochs=150, batch_size=10)

In [103]:
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [104]:
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/150
77/77 [==============================] - 1s 2ms/step - loss: 2.2253 - accuracy: 0.5938
Epoch 2/150
77/77 [==============================] - 0s 2ms/step - loss: 1.2914 - accuracy: 0.6146
Epoch 3/150
77/77 [==============================] - 0s 2ms/step - loss: 0.9809 - accuracy: 0.6172
Epoch 4/150
77/77 [==============================] - 0s 2ms/step - loss: 0.8559 - accuracy: 0.6406
Epoch 5/150
77/77 [==============================] - 0s 3ms/step - loss: 0.8111 - accuracy: 0.6276
Epoch 6/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7475 - accuracy: 0.6432
Epoch 7/150
77/77 [==============================] - 0s 2ms/step - loss: 0.7532 - accuracy: 0.6393
Epoch 8/150
77/77 [==============================] - 0s 3ms/step - loss: 0.7079 - accuracy: 0.6419
Epoch 9/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6883 - accuracy: 0.6419
Epoch 10/150
77/77 [==============================] - 0s 2ms/step - loss: 0.6698 - accuracy: 0.6654
Epoch 11/

In [105]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.720052 using {'optimizer': 'Adam'}


In [130]:
from tensorflow.keras.layers import BatchNormalization

def create_my_model(optimizer='adam', activation='relu'):
    # create model
    mymodel = Sequential()
    mymodel.add(Dense(12, input_dim=8, activation=activation))
    BatchNormalization()
    mymodel.add(Dense(8, activation=activation))
    BatchNormalization()
    mymodel.add(Dense(1, activation='sigmoid'))
    # Compile model
    mymodel.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return mymodel

In [131]:
model = KerasClassifier(build_fn=create_my_model, epochs=150, batch_size=10)

In [132]:
optimizer = ['SGD','Adadelta', 'RMSprop', 'Adagrad','Adam']
parameter_grid = dict(optimizer=optimizer)

In [141]:
from tensorflow.keras.callbacks import EarlyStopping
# Set up early stopping
early_stopping = EarlyStopping(patience=2, restore_best_weights=True ,monitor='val_loss')
grid = GridSearchCV(estimator=model, param_grid=parameter_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)

Epoch 1/150
77/77 [==============================] - 1s 5ms/step - loss: 9.6210 - accuracy: 0.6198
Epoch 2/150
77/77 [==============================] - 0s 4ms/step - loss: 2.5700 - accuracy: 0.4792
Epoch 3/150
77/77 [==============================] - 0s 4ms/step - loss: 1.3703 - accuracy: 0.4427
Epoch 4/150
77/77 [==============================] - 0s 4ms/step - loss: 1.0228 - accuracy: 0.4740
Epoch 5/150
77/77 [==============================] - 0s 4ms/step - loss: 0.8522 - accuracy: 0.4909
Epoch 6/150
77/77 [==============================] - 0s 4ms/step - loss: 0.7565 - accuracy: 0.5169
Epoch 7/150
77/77 [==============================] - 0s 4ms/step - loss: 0.7049 - accuracy: 0.6536
Epoch 8/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6706 - accuracy: 0.6745
Epoch 9/150
77/77 [==============================] - 0s 4ms/step - loss: 0.6602 - accuracy: 0.6732
Epoch 10/150
77/77 [==============================] - 0s 3ms/step - loss: 0.6546 - accuracy: 0.6758
Epoch 11/

In [142]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.736979 using {'optimizer': 'Adam'}
